In [1]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab.patches import cv2_imshow
import os

In [2]:
def extract_features(image_path):
    # Load the image
    image = cv2.imread(image_path)
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Calculate the color histogram
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    # Normalize the histogram
    features = cv2.normalize(hist, hist).flatten()
    return features

In [3]:
def calculate_similarity(query_features, dataset_features):
    # Calculate the cosine similarity between the query image features and dataset image features
    similarities = cosine_similarity([query_features], dataset_features)[0]
    return similarities

In [4]:
def main():
 # Path to the dataset directory
    dataset_path = "/content/image_dataset"
    # List to store image paths and features
    image_paths = []
    dataset_features = []

    # Loop through the dataset directory to extract features from each image
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            image_path = os.path.join(root, file)
            image_paths.append(image_path)
            # Extract features from the image
            features = extract_features(image_path)
            dataset_features.append(features)

    # Path to the query image
    query_image_path = "/content/image_dataset/circle_red488.png"
    # Extract features from the query image
    query_features = extract_features(query_image_path)

    # Calculate similarity between query image and dataset images
    similarities = calculate_similarity(query_features, dataset_features)

    # Display images that are both similar in color and shape to the query image

    for i, similarity in enumerate(similarities):
        if similarity > 0.85:
            img = cv2.imread(image_paths[i])
            contours, _ = cv2.findContours(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            is_similar_shape = False
            for contour in contours:
                perimeter = cv2.arcLength(contour, True)
                area = cv2.contourArea(contour)
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                if circularity > 0.7:
                    is_similar_shape = True
                    break
            if is_similar_shape:
                cv2_imshow(img)

if __name__ == "__main__":
    main()
